# Algoritmos de optimización - Trabajo Práctico<br>
Nombre y Apellidos: Oscar Valls Lozano  <br>
Url: https://github.com/.../03MAIR---Algoritmos-de-Optimizacion---/tree/master/TrabajoPractico<br>
Google Colab: https://colab.research.google.com/drive/8kUdmH52RVMinxWx5ADtrxu7-5eGwjK6?usp=sharing <br>
Problema:
>1. Sesiones de doblaje <br>

Descripción del problema:

Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las
tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de
doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de
grabación independientemente del número de tomas que se graben. No es posible grabar más
de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que el gasto por los
servicios de los actores de doblaje sea el menor posible. Los datos son:                                      

# Modelo
- ¿Como represento el espacio de soluciones?
- ¿Cual es la función objetivo?
- ¿Como implemento las restricciones?

1. El espacio de soluciones es una matriz de 5x6 donde las 5 filas son los días de grabación y las 6 columnas son las grabaciones que se hacen en cada día. De esta matriz se extraen los actores que actuan cada día y por ello el coste total de las grabaciones de cada día.

2. La función objetivo es la suma de los costes de las grabaciones de cada día.

3. Las restricciones de grabaciones por día ya están implementadas por las dimensiones de la matriz de actuaciones diarias.

In [2]:
import os
import numpy as np
import pandas as pd

In [5]:
#Respuesta
# 1. Se genera una ordenación aleatoria de las 30 sesiones y se agrupan en una matriz de 5 días de grabación por 6 sesiones de grabación diarias
np.random.seed(42)
grabaciones_diarias = np.arange(30)
np.random.shuffle(grabaciones_diarias)
grabaciones_diarias = np.reshape(grabaciones_diarias,newshape=(5,6))
print(grabaciones_diarias)

# 2. Se obtienen las intervenciones de actores en cada día y el coste total
actores = pd.read_excel('./datos_actores.xlsx').drop('Escena',axis='columns').to_numpy()
def get_cost(grabaciones_diarias,actores):
    coste_dias = np.zeros(grabaciones_diarias.shape[0])
    for dia in range(grabaciones_diarias.shape[0]):
        grabaciones_dia = grabaciones_diarias[dia,:]
        actores_dia = actores[grabaciones_dia,:]
        coste_dias[dia] = np.sum(np.sum(actores_dia,axis=0)>=1,axis=0)
    return coste_dias
coste_dias = get_cost(grabaciones_diarias,actores)
(np.sum(coste_dias))

[[27 15 23 17  8  9]
 [28 24 12  0  4 16]
 [ 5 13 11 22  1  2]
 [25  3 21 26 18 29]
 [20  7 10 14 19  6]]


38.0

# Análisis
- ¿Que complejidad tiene el problema?. Orden de complejidad y Contabilizar el espacio de soluciones

* El número de posibles soluciones es el número de posibles ordenaciones de las escenas que es 30! = 2.6525286e+32

# Diseño
- ¿Que técnica utilizo? ¿Por qué?

Dado el elevado número de posibles soluciones, se va a optar por el método GRASP para construir las listas de las escenas que se grabarán cada día de forma óptima.

In [16]:
# Carga de los datos
escenas = pd.read_excel('./datos_actores.xlsx').set_index('Escena').astype(int)

def get_distances(escenas_ordenadas,escenas_restantes):
    distancias = []
    for actores_escena_proxima in escenas_restantes.to_numpy():
        actores_escena_actual = escenas_ordenadas.iloc[-1].to_numpy()
        coste_adicional = int(np.sum(np.abs(actores_escena_actual - actores_escena_proxima)))
        distancias.append(coste_adicional)
    return distancias

def elegir_siguiente_escena(escenas_restantes,distancias,seed):
    escenas_restantes['distancias'] = np.array(distancias)
    lrc = escenas_restantes.nsmallest(1,'distancias')
    escenas_restantes.drop('distancias',axis='columns',inplace=True)
    siguiente_escena = lrc.sample(1,random_state=seed)
    

    return siguiente_escena

def local_search(escenas_ordenadas,escenas_restantes,seed):
    distancias = get_distances(escenas_ordenadas,escenas_restantes)
    siguiente_escena = elegir_siguiente_escena(escenas_restantes,distancias,seed).index
    return siguiente_escena

def update(escenas_ordenadas,escenas_restantes,siguiente_escena):
    escenas_ordenadas = pd.concat((escenas_ordenadas,escenas_restantes.loc[siguiente_escena,:]))
    escenas_restantes.drop(index=siguiente_escena,axis='rows',inplace=True)

    return escenas_ordenadas, escenas_restantes   

## Búsqueda multiarranque
def multiarranque(escenas,seed=42):
    np.random.seed(seed)
    soluciones = []
    # Se escoge cada una de las escenas como escena inicial
    for init_escena in range(1,len(escenas)+1):

        escenas_ordenadas = escenas.loc[[init_escena]]

        # Se inicializa un DF con las escenas no asignadas
        escenas_restantes = escenas.drop(index=escenas_ordenadas.tail(1).index,axis='rows')

        while len(escenas_ordenadas) < len(escenas):
            siguiente_escena = local_search(escenas_ordenadas,escenas_restantes,seed)
            escenas_ordenadas, escenas_restantes = update(escenas_ordenadas,escenas_restantes,siguiente_escena)
        
        soluciones.append(escenas_ordenadas)
    return soluciones


soluciones = multiarranque(escenas)

In [17]:
coste_min = 40
for escenas_ordenadas in soluciones:
    solucion = np.reshape(escenas_ordenadas.index.values,(5,6))
    coste_diario = get_cost(solucion-1,actores)
    coste_total = np.sum(coste_diario)
    if coste_total < coste_min:
        coste_min = coste_total
        mejor_solucion = escenas_ordenadas
print(f"Mejor Solucion --> Coste = {coste_min}")
mejor_solucion

Mejor Solucion --> Coste = 28.0


,Actor 1,Actor 2,Actor 3,Actor 4,Actor 5,Actor 6,Actor 7,Actor 8,Actor 9,Actor 10
Escena,,,,,,,,,,
16,0,0,0,1,0,0,0,0,0,1
25,1,1,0,1,0,0,0,0,0,1
9,1,1,0,1,0,0,0,0,0,0
6,1,1,0,1,1,0,0,0,0,0
7,1,1,0,1,1,0,0,0,0,0
1,1,1,1,1,1,0,0,0,0,0
20,1,0,1,1,1,0,0,0,0,0
2,0,0,1,1,1,0,0,0,0,0
27,0,0,0,1,1,0,0,0,0,0
